In [1]:
# Cell 1: Setup & Configuration
# --- ติดตั้ง Libraries ที่จำเป็น ---
# opendatasets: สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# timm: คลังโมเดล SOTA (State-of-the-art) รวมถึง ConvNeXt
# albumentations: สำหรับทำ Data Augmentation ที่มีประสิทธิภาพ
!pip install opendatasets --quiet
!pip install timm --quiet
!pip install albumentations --quiet
!pip install opencv-python-headless --quiet # OpenCV แบบไม่มีส่วน GUI

# --- Import Libraries หลัก ---
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tqdm.notebook import tqdm
import glob

# --- กำหนดค่า Configuration หลักของโปรเจกต์ ---
class CFG:
    # ตั้งค่าทั่วไป
    PROJECT_NAME = "Plant-Disease-Classification"
    MODEL_NAME = 'convnext_tiny' # โมเดลที่เราจะใช้จาก timm
    
    # กำหนด Path
    DATA_PATH = "./new-plant-diseases-dataset"
    
    # ตั้งค่าการเทรน
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    IMG_SIZE = 224
    BATCH_SIZE = 32
    EPOCHS = 5 # ลดจำนวน Epoch เพื่อให้รันจบเร็วในการทดลอง (ค่าที่แนะนำคือ 10-20)
    LEARNING_RATE = 1e-4
    NUM_WORKERS = 2 # จำนวน core ที่จะใช้โหลดข้อมูล
    
print(f"จะทำการเทรนบนอุปกรณ์: {CFG.DEVICE}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.3 MB/s eta 0:00:00:00:0100:01


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


จะทำการเทรนบนอุปกรณ์: cuda


In [2]:
# Cell 2: Data Acquisition
import opendatasets as od

# URL ของ Dataset บน Kaggle
dataset_url = 'https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset'

# ดาวน์โหลดข้อมูล (จะมีการถามหา username และ key จาก kaggle.json)
od.download(dataset_url)

# กำหนด Path ของข้อมูลที่ดาวน์โหลดมา
# โครงสร้าง Path: ./new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train
# เราจะใช้ Path ที่ถูกต้อง
CFG.TRAIN_PATH = os.path.join(CFG.DATA_PATH, 'New Plant Diseases Dataset(Augmented)', 'New Plant Diseases Dataset(Augmented)', 'train')
CFG.VALID_PATH = os.path.join(CFG.DATA_PATH, 'New Plant Diseases Dataset(Augmented)', 'New Plant Diseases Dataset(Augmented)', 'valid')

print(f"Train Path: {CFG.TRAIN_PATH}")
print(f"Valid Path: {CFG.VALID_PATH}")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  surasan092


Your Kaggle Key:

  ········


Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
Train Path: ./new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train
Valid Path: ./new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid


In [3]:
# Cell 3: Data Exploration

# ดึงชื่อคลาสทั้งหมดจากชื่อโฟลเดอร์ใน train path
class_names = sorted(os.listdir(CFG.TRAIN_PATH))
num_classes = len(class_names)
print(f"พบข้อมูลทั้งหมด {num_classes} คลาส")

# สร้าง DataFrame เพื่อนับจำนวนไฟล์ในแต่ละคลาสของ train และ valid set
data_counts = []
for class_name in class_names:
    train_count = len(os.listdir(os.path.join(CFG.TRAIN_PATH, class_name)))
    valid_count = len(os.listdir(os.path.join(CFG.VALID_PATH, class_name)))
    data_counts.append({'Class': class_name, 'Train Count': train_count, 'Valid Count': valid_count})

df_counts = pd.DataFrame(data_counts)
print("สรุปจำนวนข้อมูลในแต่ละคลาส:")
display(df_counts)

# เก็บชื่อคลาสและจำนวนคลาสไว้ใน CFG
CFG.CLASS_NAMES = class_names
CFG.NUM_CLASSES = num_classes


พบข้อมูลทั้งหมด 38 คลาส
สรุปจำนวนข้อมูลในแต่ละคลาส:


,Class,Train Count,Valid Count
0,Apple___Apple_scab,2016,504
1,Apple___Black_rot,1987,497
2,Apple___Cedar_apple_rust,1760,440
3,Apple___healthy,2008,502
4,Blueberry___healthy,1816,454
5,Cherry_(including_sour)___Powdery_mildew,1683,421
6,Cherry_(including_sour)___healthy,1826,456
7,Corn_(maize)___Cercospora_leaf_spot Gray_leaf_...,1642,410
8,Corn_(maize)___Common_rust_,1907,477
9,Corn_(maize)___Northern_Leaf_Blight,1908,477


In [4]:
# Cell 4: Data Augmentation
# สร้าง Pipeline สำหรับ Training Set (มีการสุ่มปรับแต่งภาพ)
train_transforms = A.Compose([
    A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# สร้าง Pipeline สำหรับ Validation/Test Set (ไม่มีการสุ่มปรับแต่ง)
valid_transforms = A.Compose([
    A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])


In [5]:
# Cell 5: Custom Dataset Class
class PlantDiseaseDataset(Dataset):
    def __init__(self, data_path, class_names, transforms=None):
        self.image_paths = glob.glob(os.path.join(data_path, '*/*.jpg')) + \
                           glob.glob(os.path.join(data_path, '*/*.JPG')) + \
                           glob.glob(os.path.join(data_path, '*/*.jpeg'))
        
        self.class_names = class_names
        self.transforms = transforms
        
        # สร้าง mapping จากชื่อคลาส (str) ไปเป็น label (int)
        self.class_to_idx = {name: i for i, name in enumerate(class_names)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        # อ่านภาพด้วย OpenCV (อ่านเป็น BGR)
        image = cv2.imread(image_path)
        # แปลงเป็น RGB ซึ่งเป็น Format ที่โมเดลส่วนใหญ่ใช้
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # ดึงชื่อคลาสจาก path
        class_name = os.path.basename(os.path.dirname(image_path))
        label = self.class_to_idx[class_name]
        
        # ทำ Augmentation ถ้ามี
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        return image, torch.tensor(label, dtype=torch.long)


In [6]:
# Cell 6: Create Datasets and DataLoaders

# สร้าง Dataset
train_dataset = PlantDiseaseDataset(CFG.TRAIN_PATH, CFG.CLASS_NAMES, transforms=train_transforms)
valid_dataset = PlantDiseaseDataset(CFG.VALID_PATH, CFG.CLASS_NAMES, transforms=valid_transforms)

# สร้าง DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.BATCH_SIZE,
    shuffle=True,
    num_workers=CFG.NUM_WORKERS
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=CFG.BATCH_SIZE,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS
)

print(f"จำนวน Training batches: {len(train_loader)}")
print(f"จำนวน Validation batches: {len(valid_loader)}")

# --- ตรวจสอบการทำงานของ DataLoader ---
# ลองดึงข้อมูล 1 batch ออกมาดู
images, labels = next(iter(train_loader))
print(f"Shape ของ image batch: {images.shape}") # (Batch Size, Channels, Height, Width)
print(f"Shape ของ label batch: {labels.shape}") # (Batch Size)


จำนวน Training batches: 2197
จำนวน Validation batches: 550
Shape ของ image batch: torch.Size([32, 3, 224, 224])
Shape ของ label batch: torch.Size([32])


In [7]:
# Cell 7: Create Model

# โหลดโมเดล ConvNeXt-tiny ที่ pre-trained บน ImageNet
# และแก้ไขจำนวนคลาสของชั้นสุดท้ายให้เท่ากับโจทย์ของเรา
model = timm.create_model(
    CFG.MODEL_NAME,
    pretrained=True,
    num_classes=CFG.NUM_CLASSES
)

# ย้ายโมเดลไปทำงานบน GPU (ถ้ามี)
model.to(CFG.DEVICE)

# แสดงจำนวนพารามิเตอร์ของโมเดล
total_params = sum(p.numel() for p in model.parameters())
print(f"Model: {CFG.MODEL_NAME}")
print(f"Total Parameters: {total_params:,}")


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Model: convnext_tiny
Total Parameters: 27,849,350


In [8]:
# Cell 8: Training and Validation Functions

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train() # ตั้งค่าโมเดลเป็น training mode
    running_loss = 0.0
    
    # ใช้ tqdm เพื่อแสดง progress bar
    for images, labels in tqdm(dataloader, desc="Training"):
        # ย้ายข้อมูลไปที่ GPU
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() # ล้างค่า gradient เก่า
        loss.backward() # คำนวณ gradient
        optimizer.step() # อัปเดตน้ำหนัก
        
        running_loss += loss.item() * images.size(0)
        
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval() # ตั้งค่าโมเดลเป็น evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    
    with torch.no_grad(): # ไม่ต้องคำนวณ gradient ตอนวัดผล
        for images, labels in tqdm(dataloader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # หาคลาสที่โมเดลทำนาย
            _, preds = torch.max(outputs, 1)
            
            running_loss += loss.item() * images.size(0)
            correct_predictions += torch.sum(preds == labels.data)
            
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item()


In [9]:
# Cell 9: The Main Training Loop

# กำหนด Loss function และ Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.LEARNING_RATE)

# ตัวแปรสำหรับเก็บผลลัพธ์ที่ดีที่สุด
best_val_acc = 0.0
best_model_path = f"{CFG.MODEL_NAME}_best.pth"

# เริ่มการเทรน
for epoch in range(CFG.EPOCHS):
    print(f"--- Epoch {epoch+1}/{CFG.EPOCHS} ---")
    
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, CFG.DEVICE)
    val_loss, val_acc = validate_one_epoch(model, valid_loader, criterion, CFG.DEVICE)
    
    print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    
    # บันทึกโมเดลที่ดีที่สุด
    if val_acc > best_val_acc:
        print(f"Validation accuracy improved from {best_val_acc:.4f} to {val_acc:.4f}. Saving model...")
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        
print("Finished Training!")
print(f"Best validation accuracy: {best_val_acc:.4f}")
print(f"Best model saved to: {best_model_path}")


--- Epoch 1/5 ---


Training:   0%|          | 0/2197 [00:00<?, ?it/s]

Validation:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch 1: Train Loss: 0.1369 | Val Loss: 0.0832 | Val Acc: 0.9745
Validation accuracy improved from 0.0000 to 0.9745. Saving model...
--- Epoch 2/5 ---


Training:   0%|          | 0/2197 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch 4: Train Loss: 0.0307 | Val Loss: 0.0198 | Val Acc: 0.9937
Validation accuracy improved from 0.9932 to 0.9937. Saving model...
--- Epoch 5/5 ---


Training:   0%|          | 0/2197 [00:00<?, ?it/s]

Validation:   0%|          | 0/550 [00:00<?, ?it/s]

Epoch 5: Train Loss: 0.0245 | Val Loss: 0.0193 | Val Acc: 0.9930
Finished Training!
Best validation accuracy: 0.9937
Best model saved to: convnext_tiny_best.pth


In [1]:
# Cell 10: Evaluation on the Validation Set

# โหลดโมเดลที่ดีที่สุดกลับมา
model.load_state_dict(torch.load(best_model_path, map_location=CFG.DEVICE))
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in tqdm(valid_loader, desc="Evaluating"):
        images = images.to(CFG.DEVICE)
        
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

print("Evaluation complete.")


NameError: name 'model' is not defined

In [ ]:
# Cell 11: Displaying Evaluation Results

# แสดง Classification Report
report = classification_report(all_labels, all_preds, target_names,CFG.CLASS_NAMES, zero_division=0)
print("--- Classification Report ---")
print(report)

# แสดง Confusion Matrix
print("\n--- Confusion Matrix ---")
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CFG.CLASS_NAMES)

# ทำให้ figure ใหญ่ขึ้นและหมุน label เพื่อให้อ่านง่าย
fig, ax = plt.subplots(figsize=(15, 15))
disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
plt.show()


In [ ]:
# Cell 12: Inference Function

def predict_one_image(image_path, model, transforms, class_names, device):
    # อ่านและแปลงภาพ
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # ทำ pre-processing
    transformed_image = transforms(image=image)['image']
    
    # เพิ่มมิติของ batch (1, C, H, W) และย้ายไป GPU
    image_tensor = transformed_image.unsqueeze(0).to(device)
    
    # ทำนายผล
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        # แปลง output (logits) เป็นความน่าจะเป็น (probabilities)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        
        # หาคลาสที่มีความน่าจะเป็นสูงสุด
        confidence, pred_idx = torch.max(probabilities, 0)
        
    predicted_class = class_names[pred_idx.item()]
    
    return predicted_class, confidence.item()

# --- ทดลองใช้งานฟังก์ชัน ---
# สุ่มภาพจาก validation set มา 1 ภาพ
sample_image_path = valid_dataset.image_paths[np.random.randint(len(valid_dataset))]

# ทำนายผล
predicted_class, confidence = predict_one_image(sample_image_path, model, valid_transforms, CFG.CLASS_NAMES, CFG.DEVICE)

# แสดงผล
img_display = cv2.imread(sample_image_path)
img_display = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)
plt.imshow(img_display)
plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.4f}")
plt.axis('off')
plt.show()


#### test

In [ ]:
# Cell 13: Setup for Submission

# --- (สมมติฐาน) กำหนดให้โฟลเดอร์ valid คือโฟลเดอร์ test ของเรา ---
CFG.TEST_PATH = CFG.VALID_PATH
print(f"Test data path is set to: {CFG.TEST_PATH}")

# ค้นหาไฟล์ภาพทั้งหมดในโฟลเดอร์ test
test_image_paths = glob.glob(os.path.join(CFG.TEST_PATH, '*/*.*'))
# ดึงมาเฉพาะชื่อไฟล์
test_filenames = [os.path.basename(p) for p in test_image_paths]

print(f"พบรูปภาพสำหรับทดสอบทั้งหมด: {len(test_filenames)} รูป")

# --- สร้างไฟล์ sample_submission.csv จำลอง ---
# เพื่อให้เห็นภาพว่า Format ที่ต้องการเป็นอย่างไร
sample_df = pd.DataFrame({
    'image': test_filenames[:5], # เอาแค่ 5 รูปแรกมาเป็นตัวอย่าง
    'label': ['' for _ in range(5)] # คอลัมน์ label จะเว้นว่างไว้ให้เราเติม
})
print("\nตัวอย่างรูปแบบของไฟล์ submission ที่ต้องการ:")
display(sample_df)


In [ ]:
# Cell 14: Create Test Dataset and DataLoader

class PlantTestDataset(Dataset):
    def __init__(self, data_path, transforms=None):
        self.image_paths = glob.glob(os.path.join(data_path, '*/*.*'))
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        # ดึงชื่อไฟล์เพื่อใช้เป็น ID
        image_id = os.path.basename(image_path)
        
        # อ่านและแปลงภาพ
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # ทำ Augmentation (ในที่นี้คือ valid_transforms)
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        return image, image_id


# สร้าง Test Dataset และ DataLoader
test_dataset = PlantTestDataset(CFG.TEST_PATH, transforms=valid_transforms)
test_loader = DataLoader(
    test_dataset,
    batch_size=CFG.BATCH_SIZE * 2, # ใช้ batch size ใหญ่ขึ้นได้ตอน inference เพราะไม่ต้องเก็บ gradient
    shuffle=False,
    num_workers=CFG.NUM_WORKERS
)

print(f"สร้าง Test Dataloader สำเร็จ มีทั้งหมด {len(test_loader)} batches")


In [ ]:
# Cell 15: Inference on Test Set

# โหลดโมเดลที่ดีที่สุดกลับมา
model.load_state_dict(torch.load(best_model_path, map_location=CFG.DEVICE))
model.eval() # *** สำคัญมาก: ตั้งค่าเป็น Evaluation Mode ***

# List สำหรับเก็บผลลัพธ์
results = []

# ไม่ต้องคำนวณ gradient
with torch.no_grad():
    for images, image_ids in tqdm(test_loader, desc="Predicting on test data"):
        images = images.to(CFG.DEVICE)
        
        # ทำนายผล
        outputs = model(images)
        _, preds_indices = torch.max(outputs, 1)
        
        # แปลง index กลับเป็นชื่อคลาส
        preds_labels = [CFG.CLASS_NAMES[i] for i in preds_indices.cpu().numpy()]
        
        # จับคู่ชื่อไฟล์กับคำทำนายแล้วเก็บใน list
        for img_id, label in zip(image_ids, preds_labels):
            results.append({'image': img_id, 'label': label})

print(f"\nทำนายผลเสร็จสิ้น! มีผลลัพธ์ทั้งหมด {len(results)} รายการ")


In [ ]:
# Cell 16: Create and Save Submission File

# แปลง list ของผลลัพธ์เป็น DataFrame
submission_df = pd.DataFrame(results)

# บันทึกเป็นไฟล์ CSV
# index=False คือการไม่เซฟคอลัมน์ index ของ DataFrame ลงในไฟล์ ซึ่งสำคัญมาก
submission_df.to_csv('submission.csv', index=False)

print("สร้างไฟล์ submission.csv สำเร็จ!")
print("ตัวอย่าง 10 แถวแรกของไฟล์:")
display(submission_df.head(10))
